In [ ]:
import cv2
import os
from operator import itemgetter
from numpy import array
import csv
import time
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from sklearn.metrics import confusion_matrix

In [ ]:
# imports all the images from a specified folder, with a specific extension
# and resizes to a specific imgHeight, imgWidth

def import_dataset(path, mode, fileExtension='.jpg', imgWidth=224, imgHeight=224):
    datasetFilenamesImages = []
    dataset = []
    print("Start importing " + mode + " images...")
    for filename in os.listdir(path):
        if filename.endswith(fileExtension): 
            completePath = os.path.join(path, filename)
            image = cv2.imread(completePath, cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, dsize=(imgHeight, imgWidth), interpolation=cv2.INTER_AREA)
            filenameImage = [filename, image]
            datasetFilenamesImages.append(filenameImage)
        else:
            continue
    
    datasetFilenamesImages = sorted(datasetFilenamesImages, key=itemgetter(0))
    for x in datasetFilenamesImages:
        dataset.append(x[1])
    
    return array(dataset)

In [ ]:
def assign_labels(path_groundtruth):
    target = []
    counter = {'NV': 0, 'MEL': 0}
    i = 0
    with open(path_groundtruth, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            if i == 0:
                i += 1
                continue

            if row[1] == '1.0': # MEL
                counter['MEL'] += 1
                target.append(0)
            elif row[2] == '1.0': # NV
                counter['NV'] += 1
                target.append(1)
    print(counter)
    file.close()
    return counter, target

In [ ]:
def plot_val_train_error(fit):
    plt.plot(fit.history['accuracy'])
    plt.plot(fit.history['val_accuracy'])
    plt.grid(True)
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

    plt.plot(fit.history['loss'])
    plt.plot(fit.history['val_loss'])
    plt.grid(True)
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

In [ ]:
def create_model_b(model = 'densenet', transferLearning=True):
    if model == 'resnet':

        from tensorflow.keras.applications.resnet import ResNet101
        from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

        if transferLearning == False:
            resnet = ResNet101(include_top=False, weights=None, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
        else:
            resnet = ResNet101(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

        model = tf.keras.Sequential(resnet)
        model.add(GlobalAveragePooling2D())
        model.add(tf.keras.layers.Dropout(0.5))
        model.add(Dense(units=2, activation="softmax"))

    elif model == 'densenet':

        from tensorflow.keras.applications.densenet import DenseNet121
        from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

        if transferLearning == False:
            densenet = DenseNet121(include_top=False, weights=None, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
        else:
            densenet = DenseNet121(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

        model = tf.keras.Sequential(densenet)
        model.add(GlobalAveragePooling2D())
        model.add(tf.keras.layers.Dropout(0.5))
        model.add(Dense(units=2, activation="softmax"))

    else:
        print("That model is not available.")
        exit(0)

    return model

In [ ]:
# Learning Rate Scheduler
def scheduler(epoch, noEpochs, learningRate):
    if epoch < int(0.5 * noEpochs):
        return learningRate
    elif epoch < int(0.75 * noEpochs):
        return learningRate / 10
    else:
        return learningRate / 100

In [ ]:
# Import Training and Validation datasets and Groundtruth of classifier b
pathTrain = '/home/ruben/Desktop/smalltrain2018'
truthTrain = '/home/ruben/Desktop/HierSmall/b/labels.csv'

x_train = import_dataset(pathTrain, 'training')
counter, y_train = assign_labels(truthTrain)

pathVal  = '/home/ruben/Desktop/smallval2018'
truthVal  = '/home/ruben/Desktop/HierSmall/val/b/labels.csv'

x_val = import_dataset(pathVal, 'validation')
counter, y_val = assign_labels(truthVal)

print("Dataset Imported")

In [ ]:
# Define training variables
noEpochs = 20
learningRate = 1e-5
noClasses = 2
batchSize = 10
modelName = 'densenet'

checkpointPath = {
    'resnet': "/home/ruben/Desktop/Small/Hier/b/resnet/cp.ckpt",
    'densenet': "/home/ruben/Desktop/Small/Hier/b/densenet/cp.ckpt"
}

n_0_float = float(counter['MEL'])
n_1_float = float(counter['NV'])
n_total = n_0_float + n_1_float
w_0 = n_total / n_0_float
w_1 = n_total / n_1_float
classWeights = {0: w_0, 1: w_1}

y_train_cat = keras.utils.to_categorical(y_train, noClasses)
y_val_cat = keras.utils.to_categorical(y_val, noClasses)

# Callback to adapt the learning rate value
schedulerLearningRate = tf.keras.callback.LearningRateScheduler(scheduler)

# Callback to save the Best Model Weights
checkpoint_dir = os.path.dirname(checkpointPath[modelName])
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpointPath[modelName],
    save_weights_only=True,
    verbose=1, 
    save_best_only=True
)

In [ ]:
# Create and fit the model
model = create_model(modelName, transferLearning=True)
model.summary()

model.compile(
    loss = keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(lr=1e-5),
    metrics = ['accuracy']
)

model.fit(
    x_train,
    y_train_cat,
    batch_size=batchSize,
    class_weight=classWeights,
    callbacks=[schedulerLearningRate, checkpoint],
    epochs = noEpochs,
    shuffle=True
    validation_data=(x_val, y_val_cat)
)

In [ ]:
# Inference Stage
model.load_weights(checkpointPath[modelName])

y_pred = model.predict_classes(x_val)
conf_matrix = confusion_matrix(y_val, y_pred)
print(conf_matrix)

score = model.evaluate(x_val, y_val_cat, verbose=1)
print(f'Val loss: {score[0]} / Val accuracy: {score[1]}')

plot_val_train_error(fit)